In [1]:
# Libraries
## Mathematics
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
from scipy.optimize import minimize_scalar

## Visualization
import matplotlib.pyplot as plt

In [2]:
# Load functions .py files
from Practical_case_functions import EM_Algorithm, KDE, sigma_log_likelihood
from Visualisation import OSB_visualisation
from Checking_simulated_boundary_functions_3 import optimal_stopping_montecarlo_3

# Data preparation

## Options dataframe

In [3]:
#df_options = pd.read_csv(r'C:\Users\User\Documents\Data\.csv', sep=',', header=0, low_memory=False)


# Visualisation final dataset
#df_options_final = []

# Define Z_1
Z_1 = np.random.normal(loc=129, scale=1, size=1000)

## Stock dataframe

In [4]:
# Read data
df_stocks = pd.read_csv(r'C:\Users\User\Documents\Data\IBM_5minPrice_ 25JUL2008 - 13DEC2018.csv', sep=',', header=0, low_memory=False)

# Extract data of interest
df_stocks["Date"] = df_stocks["Time.Interval"].str[:9]
first_date = "25JUL2008"
first_date_index = df_stocks.index[df_stocks["Date"] == first_date][0]
last_date = "08AUG2008"
last_date_index = df_stocks.index[df_stocks["Date"] == last_date][0]+78 # Add 78 that are all the intervals considered in a day
df_stocks_subset = df_stocks.loc[first_date_index:last_date_index]
days_considered = df_stocks_subset.isna().any(axis=1).sum()
df_stocks_filtered = df_stocks_subset.dropna(axis=0)

df_stocks_filtered1 = df_stocks_filtered[["Close", "Open", "High", "Low"]]
df_stocks_filtered2 = df_stocks_filtered1.copy()
df_stocks_filtered2["Middle"] = (df_stocks_filtered1["High"] + df_stocks_filtered1["Low"])/2

# Save final dataset
df_stocks_final = df_stocks_filtered2
df_stocks_final.reset_index(drop=True, inplace=True)
df_stocks_final.head()

,Close,Open,High,Low,Middle
0,129.65,129.00,129.90,128.85,129.375
1,129.35,129.65,129.67,129.20,129.435
2,129.23,129.39,129.47,129.14,129.305
3,129.23,129.22,129.36,129.15,129.255
4,129.31,129.23,129.38,129.12,129.250


## Volatility estimation

In [5]:
# Choose the stock price of the time series as a pandas Series
Z_hat = df_stocks_final["Middle"]

# Calculate logarithmic returns
log_returns = np.log(Z_hat / Z_hat.shift(1)).dropna()

# Calculate volatility (standard deviation of log returns)
volatility = log_returns.std()

# Density estimation

## EM Algorithm

In [6]:
# EM Algorithm
weights_EM, parameters_EM = EM_Algorithm(Z_1 = Z_1, n_components = "BIC", n_components_trials = 15)

C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows wi

The lowest BIC was achieved with 1 components.


C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\User\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


## KDE

In [7]:
# KDE 
weights_KDE, parameters_KDE = KDE(Z_1 = Z_1)

# Volatility estimation

In [8]:
T= df_stocks_final.shape[0]*5 # Each minute is a step of size 1

## EM Algorithm

In [9]:
# Initialize variables
tilde_z_t = df_stocks_final['Middle']
tilde_t = np.linspace(0, T, df_stocks_final.shape[0])
tilde_m = parameters_EM[0]
tilde_gamma2 = parameters_EM[1]**2
pi = weights_EM

# Maximize log-likelihood function
result = minimize_scalar(lambda s: -sigma_log_likelihood(s, tilde_z_t, tilde_t, tilde_m, tilde_gamma2, pi, T),
                         bounds=(1e-3, 15), method='bounded')

volatility_EM = result.x
print(volatility_EM)

14.999995493999162


## KDE

In [ ]:
# Initialize variables
tilde_z_t = df_stocks_final['Middle']
tilde_t = np.linspace(0, T, df_stocks_final.shape[0])
tilde_m = parameters_KDE[0]
tilde_gamma2 = parameters_KDE[1]**2
pi = weights_KDE

# Maximize log-likelihood function
result = minimize_scalar(lambda s: -sigma_log_likelihood(s, tilde_z_t, tilde_t, tilde_m, tilde_gamma2, pi, T),
                         bounds=(1e-3, 15), method='bounded')

volatility_KDE = result.x
print(volatility_KDE)

# Scaling

## EM Algorithm

## KDE

# Boundary obtention

## EM Algorithm

In [ ]:
# OSB estimation
# Temporal mesh
mesh = 10
t_mesh = np.linspace(0, 1, mesh) # Temporal grid

# Spatial grid
L = 12500

# Boundary
np.random.seed(123)
value_function_EM, X_vals_EM = optimal_stopping_montecarlo_3(mu = "continuous", weights = weights_EM, parameters = parameters_EM, N = mesh, a = min(Z_1)-1, b = max(Z_1)+1, L = L, M = 4500)

In [ ]:
# Plotting
OSB_visualisation(N = mesh, value_function = value_function_EM, X_vals = X_vals_EM, title = "Practical case using EM algorithm", name = "Practical_case_EM.png")

## KDE

In [ ]:
# OSB estimation
# Temporal mesh
mesh = 10
t_mesh = np.linspace(0, 1, mesh) # Temporal grid

# Spatial grid
L = 12500

# Boundary
np.random.seed(123)
value_function_KDE, X_vals_KDE = optimal_stopping_montecarlo_3(mu = "continuous", weights = weights_KDE, parameters = parameters_KDE, N = mesh, a = min(Z_1)-1, b = max(Z_1)+1, L = L, M = 4500)

In [ ]:
# Plotting
OSB_visualisation(N = mesh, value_function = value_function_KDE, X_vals = X_vals_KDE, title = "Practical case using KD", name = "Practical_case_KDE.png")